<a href="https://colab.research.google.com/github/MuayThaiLegz/PracticeCrazy/blob/main/EmailSort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install xformers
!pip install transformers
!pip install torch
!pip install --upgrade accelerate
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.7 MB/s eta 0:00:00


In [2]:
# import random

# def generate_dummy_data(num_records=10000):
#     # Sample email addresses for company employees and contractors
#     company_emails = [f'user{str(i)}@xyz.com' for i in range(50)]
#     lawfirm_emails = [f'lawyer{str(i)}@lawfirm.com' for i in range(50)]
#     # Sample projects/matters/legal cases
#     projects = [f'Case {i}' for i in range(50)] + [f'Project {i}' for i in range(50)] + [f'Matter {i}' for i in range(50)]
#     # Sample subjects related to legal matters
#     subjects = ["Deposition Next Week", "Meeting Reminder", "Contract Review", "Acquisition Discussion", "Next Steps in Case"]

#     data = []
#     for _ in range(num_records):
#         from_email = random.choice(company_emails)
#         to_email = random.choice(lawfirm_emails)
#         subject = random.choice(subjects)
#         content = f"This email is regarding {subject.lower()} related to {random.choice(projects)}. Please review the attached documents."
#         # Simulating the 94% probability for a specific scenario
#         if from_email == 'mary@xyz.com' and to_email == 'tom@lawfirm.com' and subject == "Deposition Next Week":
#             project_match = 'Zebra v. Dog Case' if random.random() < 0.94 else 'Unmatched'
#         else:
#             project_match = random.choice(projects) if random.random() > 0.10 else 'Unmatched'

#         record = {
#             'from_email': from_email,
#             'to_email': to_email,
#             'subject': subject,
#             'content': content,
#             'project_match': project_match
#         }
#         data.append(record)

#     return data

# # Example usage
# dummy_data = generate_dummy_data()
# print(dummy_data[0])  # Print the first record


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
import torch

from sklearn.model_selection import train_test_split

In [4]:
from nltk.corpus import wordnet as wn
import nltk

nltk.download('wordnet')

def synonym_replace(word):
    synsets = wn.synsets(word)
    if synsets:
        synonyms = set([lemma.name() for syn in synsets for lemma in syn.lemmas()])
        synonyms.discard(word)
        if synonyms:
            return random.choice(list(synonyms))
    return word

def augment_text(text):
    words = text.split()
    for i in range(len(words)):
        if random.random() < 0.05:  # 5% chance to replace a word with a synonym
            words[i] = synonym_replace(words[i])
    return " ".join(words)

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
import random
from datetime import datetime

def generate_emails(num_emails=350000, seed_value=None):
    senders = [
        "mary@xyz.com", "john@abc.com", "sara@company.com", "mark@contractor.com", "lia@exporter.com",
        "paul@legal.com", "emma@finance.org", "lisa@audit.net", "george@compliance.com", "olivia@tax.org"]

    recipients = [
        "tom@lawfirm.com", "jane@lawfirm.com", "dom@lawfirm.com", "jill@lawfirm.com",
        "mike@lawfirm.com", "nina@lawfirm.com", "chris@lawfirm.com", "amy@lawfirm.com"]

    subjects = [
        "deposition next week", "meeting tomorrow", "contract review", "acquisition discussion", "next steps in case",
        "urgent: contract amendments", "review meeting on Friday", "new case filed", "project update", "legal compliance check"]

    cc_bcc_list = [
        "manager@lawfirm.com",
        "assistant@lawfirm.com, supervisor@lawfirm.com",
        "legal_team@lawfirm.com",
        "", # No CC or BCC
        "partner@lawfirm.com; associate@lawfirm.com"]

    timestamps = [
        "%Y-%m-%d %H:%M:%S",
        "%A, %d %B %Y %H:%M:%S",
        "%d/%m/%Y %H:%M",
        "%m/%d/%Y %I:%M %p"]

    matters = [
        "Zebra v. Dog case", "ABC project", "DEF case", "XYZ project", "Ape v. Monster case",
        "GHI project", "JKL case", "MNO project", "PQR case", "STU project"]

    body_templates = [
        "Hi {recipient_name},\n\nPlease find the attached documents for {matter}. Let's discuss further on {date}.\n\nBest, {sender_name}",
        "Dear {recipient_name},\n\nI would like to update you on {matter}. Could we schedule a call for {date}?\n\nSincerely, {sender_name}",
        "Hello {recipient_name},\n\n{matter} is progressing well. Please review the attached files and let me know your thoughts.\n\nThanks, {sender_name}",
        "{recipient_name},\n\nThe next meeting for {matter} is scheduled for {date}. Please confirm your availability.\n\nRegards, {sender_name}",
        "{recipient_name},\n\nWe need to discuss the recent developments in {matter}. Can we set up a call for {date}?\n\nYours, {sender_name}"]

    if seed_value is not None:
        random.seed(seed_value)

    emails = []
    labels = []

    for _ in range(num_emails):
        sender = random.choice(senders)
        recipient = random.choice(recipients)
        subject = random.choice(subjects)
        matter = random.choice(matters)
        body_template = random.choice(body_templates)
        cc_bcc = random.choice(cc_bcc_list)
        timestamp = datetime.now().strftime(random.choice(timestamps))

        sender_name = sender.split("@")[0]
        recipient_name = recipient.split("@")[0]
        date = random.choice(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"])

        body = body_template.format(
            recipient_name=recipient_name.capitalize(),
            sender_name=sender_name.capitalize(),
            matter=matter,
            date=date
        )
        # Please make sure to define the augment_text function elsewhere in your code or remove this line if not needed
        augmented_body = augment_text(body)
        email_content = f"From: {sender}\nTo: {recipient}\nCC/BCC: {cc_bcc}\nTimestamp: {timestamp}\nSubject: {subject}\n\n{augmented_body}\n..."
        emails.append(email_content)
        labels.append(matter)

    return emails, labels

emails, labels = generate_emails()


In [6]:
https://github.com/openai/openai-cookbook/blob/main/examples/Fine-tuned_classification.ipynb

SyntaxError: ignored

In [7]:
home_email = [
    """Declaration for review and signature
Inbox

Marcela Knaup
Attachments
Tue, Aug 8, 4:50 PM (2 days ago)
to Mayra, me

Hi Mayra

attached you will find your declaration in Javier's case.  you will notice that they are some parts that need to be completed (some dates).  You can add the information yourself or give me the info and I will add it to your declaration together with any changes that need to be made.  Eventually, send me the declaration with your changes and signature.

At this stage the entire application is done, once you send me the declaration I will send the one for Javier (because I may need to add your changes to his declaration) and then we'll send the application to USCIS.

We need to send this application this month, hopefully in the next few days because Javier prepaid the fees of the National Visa Center and the payment is a deadline to send the application no later than this month.

I am sending the last invoice for the fees of the waiver, last cost and my last invoice of the waiver.  PLEASE SEND AT LEAST THE COSTS OF THE WAIVER $715.  That payment has to be made by electronic transfer or leave a check or cash at the office.  You can later pay my fees with a credit card if needed.

Please let me know if you have any questions, the waiver is done, we'll send it as soon as we have your signatures and the payment of the fees.

thank you! m/

Marcela B. Knaup, Esq. LLM
Law Offices of Marcela B. Knaup

2950 Buskirk Ave. Ste. 300
Walnut Creek, California, 94597
Te: 925-407-2149
M:  415-409-9229
knauplegal.com

Confidentiality Note: This message and any accompanying attachments contain information from the law firm “Law Offices of Marcela B. Knaup”.  It may contain information which is strictly confidential, subject to attorney-client privilege, or otherwise privileged.  The information is intended for the individual(s) or organization(s) named above. If you have received this email in error, please reply and notify the sender (only) and delete the message.  Unauthorized interception of this e-mail is a violation of United States Federal criminal law.


Law Offices of Marcela B. Knaup
T:  925-407-2149 | M:  415-409-9229
2950 Buskirk Ave. Ste. 300
Walnut Creek, CA 94597
knauplegal.com

Confidentiality Note: This message and any accompanying attachments contain information from the law firm “Law Offices of Marcela B. Knaup”.  It may contain information which is strictly confidential, subject to attorney-client privilege, or otherwise privileged.  The information is intended for the individual(s) or organization(s) named above. If you have received this email in error, please reply and notify the sender (only) and delete the message.  Unauthorized interception of this e-mail is a violation of United States Federal criminal law.


Please consider your environmental responsibility. Before printing this e-mail message, ask yourself whether you really need a hard copy."""
]


In [8]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(p):
    pred_labels = np.argmax(p.predictions, axis=1)
    true_labels = p.label_ids
    accuracy = accuracy_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels, average='macro')
    recall = recall_score(true_labels, pred_labels, average='macro')
    f1 = f1_score(true_labels, pred_labels, average='macro')
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

def compute_metricsdos(p):
    pred_labels = np.argmax(p.predictions, axis=1)
    true_labels = p.label_ids
    accuracy = accuracy_score(true_labels, pred_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='weighted')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }





In [9]:
from sklearn.preprocessing import LabelEncoder
import torch

emails, labels = generate_emails()

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
inputs = tokenizer(emails, truncation=True, padding=True, max_length=512, return_tensors="pt")

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

train_idx, val_idx = train_test_split(range(len(encoded_labels)), test_size=0.2, random_state=42)

# Use the indices to create training and validation inputs

train_inputs = {key: val[train_idx] for key, val in inputs.items()}

val_inputs = {key: val[val_idx] for key, val in inputs.items()}

train_labels = encoded_labels[train_idx]

val_labels = encoded_labels[val_idx]


class EmailDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: self.inputs[key][idx] for key in self.inputs}
        item['labels'] = self.labels[idx]
        return item

train_dataset = EmailDataset(train_inputs, train_labels)
val_dataset = EmailDataset(val_inputs, val_labels)

# Model Training
num_classes = len(set(labels)) # Number of unique project/matter labels
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_classes)


training_args = TrainingArguments(
    output_dir='./BertForSequenceClassificationresults',
    num_train_epochs=2,
    per_device_train_batch_size=128,
    warmup_steps=300,

    weight_decay=0.05,
    save_total_limit=3,
    save_steps=1500,  # Save a checkpoint every 500 steps
    report_to=[], # Disable all integrations (e.g., TensorBoard, WandB, etc.)
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()



KeyboardInterrupt: ignored

In [16]:
results = trainer.evaluate()
print(results)

{'eval_loss': 8.749494736548513e-05, 'eval_accuracy': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_runtime': 136.5138, 'eval_samples_per_second': 512.769, 'eval_steps_per_second': 64.096, 'epoch': 2.0}


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer

# Assuming generate_emails returns a list of emails and corresponding labels
emails, labels = generate_emails()

# Tokenize emails using RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
inputs = tokenizer(emails, truncation=True, padding=True, max_length=512, return_tensors="pt")

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Get the indices of training and validation sets
train_idx, val_idx = train_test_split(range(len(encoded_labels)), test_size=0.3, random_state=42)

# Use the indices to create training and validation inputs
train_inputs = {key: val[train_idx] for key, val in inputs.items()}
val_inputs = {key: val[val_idx] for key, val in inputs.items()}
train_labels = encoded_labels[train_idx]
val_labels = encoded_labels[val_idx]

class EmailDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: self.inputs[key][idx] for key in self.inputs}
        item['labels'] = self.labels[idx]
        return item

train_dataset = EmailDataset(train_inputs, train_labels)
val_dataset = EmailDataset(val_inputs, val_labels)

# Model Training using RoBERTa
num_classes = len(set(labels)) # Number of unique project/matter labels
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=num_classes)

training_args = TrainingArguments(
    output_dir='./RobertaForSequenceClassificationresults',
    num_train_epochs=2,
    per_device_train_batch_size=16,
    warmup_steps=250,
    weight_decay=0.1,
    save_total_limit=3,
    save_steps=10000,  # Save a checkpoint every 15000 steps
    report_to=[], # Disable all integrations (e.g., TensorBoard, WandB, etc.)
)

roberta_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

roberta_trainer.train()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.468600
1000,0.000800
1500,0.000300
2000,0.000200
2500,0.000100


In [ ]:
resultsRoberta = roberta_trainer.evaluate()
print(resultsRoberta)

In [6]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch

# Generate emails and labels
emails, labels = generate_emails()

# Tokenize emails
# tokenizer = BertTokenizer.from_pretrained("lexlms/legal-roberta-large")
# inputs = tokenizer(emails, truncation=True, padding=True, max_length=512, return_tensors="pt")
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("lexlms/legal-roberta-large")
inputs = tokenizer(emails, truncation=True, padding=True, max_length=512, return_tensors="pt")

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split into training and validation sets
train_idx, val_idx = train_test_split(range(len(encoded_labels)), test_size=0.2, random_state=42)
train_inputs = {key: val[train_idx] for key, val in inputs.items()}
val_inputs = {key: val[val_idx] for key, val in inputs.items()}
train_labels = encoded_labels[train_idx]
val_labels = encoded_labels[val_idx]

# Create datasets
class EmailDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: self.inputs[key][idx] for key in self.inputs}
        item['labels'] = self.labels[idx]
        return item

train_dataset = EmailDataset(train_inputs, train_labels)
val_dataset = EmailDataset(val_inputs, val_labels)

# Model Training
num_classes = len(set(labels))
model = BertForSequenceClassification.from_pretrained("lexlms/legal-roberta-large", num_labels=num_classes)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=32,
    warmup_steps=100,
    weight_decay=0.5,
    save_total_limit=3, # Limit the number of saved checkpoints
    save_steps=10000,  # Save a checkpoint every 15,000 steps
    report_to=[], # Disable all integrations
)

legal_roberta_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

legal_roberta_trainer.train()


You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at lexlms/legal-roberta-large and are newly initialized: ['encoder.layer.16.attention.output.dense.weight', 'encoder.layer.10.intermediate.dense.weight', 'encoder.layer.23.output.LayerNorm.weight', 'encoder.layer.14.attention.self.query.bias', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.7.attention.self.key.bias', 'encoder.layer.1.attention.self.key.weight', 'encoder.layer.23.output.dense.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.8.attention.self.key.bias', 'encoder.layer.7.intermediate.dense.bias', 'encoder.layer.16.attention.self.query.bias', 'encoder.layer.9.attention.self.key.bias', 'encoder.layer.14.output.dense.bias', 'encoder.layer.1.output.LayerNorm.weight', 'classifier.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.4.attention.output.LayerNorm.weight', 'encoder.layer.9.attention.output.dense.weight', 'encoder.layer.2.attention.se

NameError: ignored

In [ ]:





legal_roberta = legal_roberta_trainer.evaluate()
print(resultsRoberta)

In [ ]:
def predict_email(email_content):
    inputs = tokenizer([email_content], truncation=True, padding=True, max_length=512, return_tensors="pt")
    prediction = model(**inputs)
    predicted_label_idx = np.argmax(prediction.logits.detach().numpy(), axis=1)
    predicted_label = label_encoder.inverse_transform(predicted_label_idx)
    return predicted_label[0]

# Example usage of prediction function
home_email_label = predict_email(home_email[0])
print("Predicted Label:", home_email_label)

In [ ]:
# Save the model, tokenizer, and label encoder
from transformers import BertTokenizer, BertForSequenceClassification
import pickle


import pickle
model_path = "./saved_model"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)
with open(model_path + "/label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)


# Load the model, tokenizer, and label encoder
model_path = "./saved_model"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)
with open(model_path + "/label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)



In [ ]:
# Tokenize the email
input_email = tokenizer(home_email, truncation=True, padding=True, max_length=512, return_tensors="pt")

# Make the prediction
with torch.no_grad():
    output = model(**input_email)

# Extract the predicted label index
predicted_label_index = torch.argmax(output.logits, dim=1).item()

# Use the label encoder to get the human-readable label
predicted_label = label_encoder.inverse_transform([predicted_label_index])[0]

print(f"The predicted label for the email is: {predicted_label}")


def predict_email(email_text):
    # Tokenize the input email
    inputs = tokenizer(email_text, truncation=True, padding=True, return_tensors="pt")

    # Get the model's predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted class label
    predicted_class = torch.argmax(outputs.logits, dim=1).item()

    # Decode the predicted class label to the original label
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]

    return predicted_label

# Example usage
email_text = home_email
prediction = predict_email(email_text)
print(f"The email is related to: {prediction}")



In [ ]:
from torch.utils.data import DataLoader

def predict(emails, model, tokenizer, label_encoder):
    # Tokenize the new emails
    inputs = tokenizer(emails, truncation=True, padding=True, max_length=512, return_tensors="pt")

    # Create a DataLoader for the new emails
    dataset = torch.utils.data.TensorDataset(inputs['input_ids'], inputs['attention_mask'])
    dataloader = DataLoader(dataset, batch_size=16)

    # Make predictions
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask = batch
            outputs = model(**inputs)
            logits = outputs.logits
            predictions.extend(torch.argmax(logits, dim=1).tolist())

    # Decode predictions to label names
    predicted_labels = label_encoder.inverse_transform(predictions)

    return predicted_labels


# Make predictions
predicted_labels = predict(home_email, model, tokenizer, label_encoder)

# Print results
for email, label in zip(home_email, predicted_labels):
    print(f"Email: {email[:50]}... -> Predicted Label: {label}")


In [ ]:
results

In [ ]:
pd.DataFrame(evaluation_results.values(), index = evaluation_results.keys())


In [ ]:
# Save the model
model_path = "./saved_model"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)


In [ ]:
# from fastapi import FastAPI
# from transformers import BertForSequenceClassification, BertTokenizer
# import torch

# app = FastAPI()

# model_path = "./saved_model"
# tokenizer = BertTokenizer.from_pretrained(model_path)
# model = BertForSequenceClassification.from_pretrained(model_path)

# @app.post("/predict")
# def predict(email: str):
#     inputs = tokenizer([email], truncation=True, padding=True, return_tensors="pt")
#     outputs = model(**inputs)
#     prediction = torch.argmax(outputs.logits, dim=1)
#     # Translate prediction to label
#     predicted_label = label_encoder.inverse_transform([prediction.item()])[0]
#     return {"prediction": predicted_label}


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer
from transformers import Trainer, TrainingArguments

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_classes)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()

In [ ]:
Email,Label
"Email text 1","Project 1"
"Email text 2","Project 2"
...


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer


# Tokenize emails
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
inputs = tokenizer(emails.tolist(), truncation=True, padding=True)

# Split into training and validation sets
train_inputs, val_inputs, train_labels, val_labels = train_test_split(inputs, labels, test_size=0.2, random_state=42)

from torch.utils.data import Dataset

class EmailDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = EmailDataset(train_inputs, train_labels)
val_dataset = EmailDataset(val_inputs, val_labels)


from transformers import BertForSequenceClassification, TrainingArguments, Trainer

num_classes = len(set(labels)) # Number of unique project/matter labels

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_classes)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset, # Add validation dataset
)

trainer.train()
